In [1]:
# Import PySpark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder.appName("Capstone Analytics Samples").getOrCreate()

In [2]:
fact_immigration = spark.read.parquet('tables/fact_immigration')
dim_immigrant_person = spark.read.parquet('tables/immigrant_dimension_indiv')
dim_city = spark.read.parquet('tables/city_dimension')
dim_time = spark.read.parquet('tables/dimension_time')

In [3]:
# tempory views for the queries

fact_immigration.createOrReplaceTempView("fact_immigration_table")
dim_immigrant_person.createOrReplaceTempView("immigrant_table")
dim_city.createOrReplaceTempView("dim_city_table")
dim_time.createOrReplaceTempView("dim_time_table")

In [4]:
spark.sql("""
    SELECT count(*) FROM fact_immigration_table
""").show()

+--------+
|count(1)|
+--------+
|  219268|
+--------+



Question: What city has the highest number of immigrants? and what might be a factor in determining where to migrate to?

My theory is, Immigrants are more likely to move to a city where there are lots of other Immigrants like themselves e.g Family.

To answer this question, we combine the city demographics data with the immigration fact data and order it by foreign_born to see the highest number of foreign borns and then sort by city to see the cities with the most immigrants

In [7]:
spark.sql("""
    SELECT *
    FROM
        (SELECT DISTINCT
            f.i94port, 
            COUNT(f.cicid) AS immigrants
        FROM fact_immigration_table f
        GROUP BY f.i94port
        ORDER BY immigrants DESC
        LIMIT 10
        ) AS immigrants_by_port
    JOIN dim_city_table c
        ON immigrants_by_port.i94port = c.city_code
    ORDER BY immigrants DESC
""").dropDuplicates().show()

+-------+----------+---------------+----------+----------+---------------+----------------+------------+---------------------+----------+---------+
|i94port|immigrants|      city_name|     state|median_age|male_population|total_population|foreign_born|average_householdsize|state_code|city_code|
+-------+----------+---------------+----------+----------+---------------+----------------+------------+---------------------+----------+---------+
|    NYC|     32779|       New York|  New York|      36.0|        4081698|         4468707|     8550405|               156961|   3212500|      NYC|
|    MIA|     22660|          Miami|   Florida|      40.4|         215840|          225149|      440989|                 7233|    260789|      MIA|
|    LOS|     20983|    Los Angeles|California|      35.0|        1958998|         2012898|     3971896|                85417|   1485425|      LOS|
|    ORL|     12332|        Orlando|   Florida|      33.1|         130940|          139977|      270917|        